# Model Fine Tuning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install Dependencies
!pip install torch torchvision torchaudio --quiet
!pip install numpy pandas scipy scikit-learn matplotlib seaborn --quiet

In [ ]:
dataset_path = '/content/drive/MyDrive/Borsa di ricerca/Post First Paper/Paper Activity recognition/Code Research/HAR/capture24_dataset'

In [ ]:
# Imports
import os, numpy as np, torch
import torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader

In [ ]:
class NPYDataset(Dataset):
    def __init__(self, x_path, y_path):
        # Apri in memmap: non carica tutto in RAM, ma mappa sul disco
        self.X = np.load(x_path, mmap_mode='r')   # shape: (N, 3, T)
        self.Y_str = np.load(y_path, mmap_mode='r')  # array di stringhe

        uniques = np.unique(self.Y_str)
        self.class2idx = {c: i for i, c in enumerate(uniques)}

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Leggi un singolo sample
        x = self.X[idx].astype(np.float32)# (3, T)
        y_str = self.Y_str[idx]
        y = self.class2idx[y_str]
        return torch.from_numpy(x), torch.tensor(y, dtype=torch.long)

In [ ]:
# === Parametri ===
x_path = f'{dataset_path}/X.npy'
y_path = f'{dataset_path}/Y.npy'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32

ds = NPYDataset(x_path, y_path)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
epochs = 10
lr = 1e-3

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Carica modello SSL (solo per estrarre la dimensione)
repo = 'OxWearables/ssl-wearables'
tmp_model = torch.hub.load(repo, 'harnet10', class_num=5, pretrained=True, trust_repo=True)
tmp_model.eval().to(device)

# Dummy input: batch=1, 3 canali, 300 time steps
dummy = torch.randn(1, 3, 300, device=device)

# Estraggo feature
with torch.no_grad():
    feat = tmp_model.feature_extractor(dummy)

feat_dim = feat.shape[1]
print("Embedding dimension:", feat_dim)

Using cache found in /root/.cache/torch/hub/OxWearables_ssl-wearables_main


131 Weights loaded
Embedding dimension: 1024


In [ ]:
# === Modello ===
repo = 'OxWearables/ssl-wearables'
num_classes = len(ds.class2idx)
model = torch.hub.load(repo, 'harnet10', class_num=num_classes, pretrained=True)
for p in model.feature_extractor.parameters(): p.requires_grad = False
model.classifier = nn.Linear(feat_dim, num_classes)
model.to(device)

Using cache found in /root/.cache/torch/hub/OxWearables_ssl-wearables_main


131 Weights loaded


Resnet(
  (feature_extractor): Sequential(
    (layer1): Sequential(
      (0): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      (1): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (relu): ReLU(inplace=True)
      )
      (2): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(6

In [ ]:
optimizer = optim.Adam(model.classifier.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(1, epochs + 1):
    model.train()
    total_loss = correct = total = 0
    for xb, yb in loader:
        xb = xb.permute(0, 2, 1)
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * xb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    print(f"Epoch {epoch}: Loss={total_loss/total:.4f}, Acc={correct/total:.4f}")

Epoch 1: Loss=0.5320, Acc=0.8023
Epoch 2: Loss=0.5184, Acc=0.8074
Epoch 3: Loss=0.5163, Acc=0.8084
Epoch 4: Loss=0.5146, Acc=0.8089
Epoch 5: Loss=0.5142, Acc=0.8092
Epoch 6: Loss=0.5136, Acc=0.8092
Epoch 7: Loss=0.5128, Acc=0.8097
Epoch 8: Loss=0.5130, Acc=0.8102
Epoch 9: Loss=0.5129, Acc=0.8093
Epoch 10: Loss=0.5120, Acc=0.8099


In [ ]:
 # Salva i pesi su Drive
out_path = f'/content/drive/MyDrive/Borsa di ricerca/Post First Paper/Paper Activity recognition/Code Research/HAR/finetuned_harnet10.pth'
torch.save(model.state_dict(), out_path)
print("Modello salvato in", out_path)

Modello salvato in /content/drive/MyDrive/Borsa di ricerca/Post First Paper/Paper Activity recognition/Code Research/HAR/finetuned_harnet10.pth


# Classification of our data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
db_path = '/content/drive/MyDrive/Borsa di ricerca/Post First Paper/Paper Activity recognition/Code Research/HAR/health_data.db'

In [18]:
Y_str = np.load('/content/drive/MyDrive/Borsa di ricerca/Post First Paper/Paper Activity recognition/Code Research/HAR/capture24_dataset/Y.npy')  # update the path accordingly

# Get unique class names and build mapping
unique_classes = np.unique(Y_str)
class2idx = {cls: idx for idx, cls in enumerate(unique_classes)}
idx2class = {idx: cls for cls, idx in class2idx.items()}

print("idx2class:", idx2class)

idx2class: {0: np.str_('light'), 1: np.str_('moderate-vigorous'), 2: np.str_('sedentary'), 3: np.str_('sleep')}


In [2]:
import sqlite3
from scipy import signal
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [6]:
def load_acc(db_path):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(
        "SELECT x_value, y_value, z_value, timestamp FROM accelerometer ORDER BY timestamp",
        conn
    )
    conn.close()
    return df

def resample_30hz(data, timestamps):
    ts = pd.to_datetime(timestamps)
    diffs = np.diff(ts.values).astype('timedelta64[ms]').astype(float)
    rate = 1000. / diffs.mean()
    if abs(rate - 30) < 1: return data
    factor = 30. / rate
    N = int(len(data) * factor)
    out = np.zeros((N,3))
    for i in range(3):
        out[:,i] = signal.resample(data[:,i], N)
    return out

def make_windows(acc_data, win_size=300, overlap=0.5):
    step = int(win_size * (1-overlap))
    W = []
    for i in range(0, len(acc_data)-win_size+1, step):
        W.append(acc_data[i:i+win_size])
    return np.stack(W)

In [7]:
df = load_acc(db_path)
acc = df[['x_value','y_value','z_value']].values
ts  = df['timestamp']
acc30 = resample_30hz(acc, ts)
wins = make_windows(acc30, win_size=300, overlap=0.5)

In [8]:
print("Windows generated:", wins.shape)  # (n_windows, 300, 3)

wins = np.transpose(wins, (0,2,1))

Windows generated: (199, 300, 3)


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

repo = 'OxWearables/ssl-wearables'
num_classes = 4  # Capture-24 ha 4 classi
model = torch.hub.load(repo, 'harnet10', class_num=num_classes, pretrained=True, trust_repo=True)

feat_dim = model.feature_extractor(torch.randn(1,3,300,device=device)).shape[1]
for p in model.feature_extractor.parameters(): p.requires_grad=False
model.classifier = nn.Linear(feat_dim, num_classes)
model.to(device)

Downloading: "https://github.com/OxWearables/ssl-wearables/zipball/main" to /root/.cache/torch/hub/main.zip


131 Weights loaded


Resnet(
  (feature_extractor): Sequential(
    (layer1): Sequential(
      (0): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      (1): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (relu): ReLU(inplace=True)
      )
      (2): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(6

In [11]:
ckpt = '/content/drive/MyDrive/Borsa di ricerca/Post First Paper/Paper Activity recognition/Code Research/HAR/finetuned_harnet10.pth'

sd = torch.load(ckpt, map_location=device)
model.load_state_dict(sd, strict=False)
model.eval()

Resnet(
  (feature_extractor): Sequential(
    (layer1): Sequential(
      (0): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
      (1): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (relu): ReLU(inplace=True)
      )
      (2): ResBlock(
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False, padding_mode=circular)
        (conv2): Conv1d(6

In [12]:
batch = torch.from_numpy(wins).float().to(device)
preds = []
with torch.no_grad():
    for i in range(0, len(batch), 256):
        out = model(batch[i:i+256])
        preds.append(out.argmax(1).cpu().numpy())
preds = np.concatenate(preds, axis=0)
print("Predizioni effettuate:", preds.shape)

Predizioni effettuate: (199,)


In [19]:
step = int(300 * 0.5)
start_time = pd.to_datetime(ts.iloc[0])
resampled_times = pd.date_range(start=start_time, periods=len(acc30), freq='33.333ms')
start_indices = np.arange(0, len(acc30)-300+1, step)
df_labels = pd.DataFrame({
    'start_time': resampled_times[start_indices],
    'end_time': resampled_times[start_indices + 300],
    'predicted_class': preds
})

df_labels['predicted_class_str'] = df_labels['predicted_class'].map(idx2class)

In [20]:
out_csv = '/content/drive/MyDrive/Borsa di ricerca/Post First Paper/Paper Activity recognition/Code Research/HAR/health_data_labels.csv'
df_labels.to_csv(out_csv, index=False)
print("Labels salvate in", out_csv)

Labels salvate in /content/drive/MyDrive/Borsa di ricerca/Post First Paper/Paper Activity recognition/Code Research/HAR/health_data_labels.csv
